# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [68]:
# Import dependencies
from pymongo import MongoClient
import pymongo
import pprint


In [7]:
# Create an instance of MongoClient
client = MongoClient("mongodb://localhost:27017/")


In [8]:
# check database exist or not
print('uk_food' in client.list_database_names())

True


In [9]:
# assign the uk_food database to a variable name
mydb = client['uk_food']
mycol = mydb['establishments']


In [10]:
# review the collections in our new database
collections = mydb.list_collection_names()
print(collections)

['establishments']


In [11]:
# assign the collection to a variable
mycol = mydb['establishments']


In [12]:
# review a document in the establishments collection
result = mycol.find_one({"LocalAuthorityName": "Dover"})
print(result)


{'_id': ObjectId('640b6189783563aa893fff01'), 'FHRSID': 254719, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000069980', 'BusinessName': 'Refreshment Kiosk', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'The Bay', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DY', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-03-24T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'}, 'RightToReply': '', 'Distance': 4587.347174863443, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'total

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [69]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant",
    "AddressLine1": "123 Main Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 9LA",
    "RatingValue": "",
    "RatingKey": "",
    "RatingDate": "",
    "LocalAuthorityName": "Greenwich",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "",
    "geocode": {
        "longitude": 1.18590330311706,
        "latitude": 51.0783519967077
    }
}



In [70]:
# Insert the new restaurant into the collection
result = mycol.insert_one(new_restaurant)
print(result)


In [15]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print("New restaurant inserted with id:", result.inserted_id)
else:
    print("Insertion failed!")


New restaurant inserted with id: 640c137551adbe78e703cd8c


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [28]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
results = mycol.find(query, projection)
# Print the results
for result in results:
    print(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaur

3. Update the new restaurant with the `BusinessTypeID` you found.

In [33]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to match the new restaurant by name
filter = {"name": "Penang Flavours"}

update = {"$set": {"BusinessTypeID": 2}}
result = mycol.update_one(filter, update)





In [40]:
# Confirm that the new restaurant was updated
if result.matched_count > 0:
    print("Restaurant updated with new BusinessTypeID.")
else:
    print("Update failed!")

print(result.raw_result)


Restaurant updated with new BusinessTypeID.
{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [41]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
count = mycol.count_documents(query)
print(count)


994


In [42]:
# Delete all documents where LocalAuthorityName is "Dover"
deleted = mycol.delete_many(query)
print(deleted.deleted_count, "Documents Deleted")


994 Documents Deleted


In [46]:
# Check if any remaining documents include Dover
check = mycol.count_documents(query)

# Check if any documents were returned
if check > 0:
    print("There are still documents with Dover as the LocalAuthorityName.")
else:
    print("All documents with Dover as the LocalAuthorityName have been deleted.")

All documents with Dover as the LocalAuthorityName have been deleted.


In [51]:
# Check that other documents remain with 'find_one'
query = {"LocalAuthorityName": {"$ne": "Dover"}}
result = mycol.find_one(query)
print(result.values())

if result is not None:
    print("At least one document with LocalAuthorityName other than 'Dover' exists.")
else:
    print("All documents with LocalAuthorityName 'Dover' have been deleted.")

dict_values([ObjectId('640b6189783563aa894001e8'), 1043695, 0, 'PI/000073616', 'The Pavilion', 'Restaurant/Cafe/Canteen', 1, 'East Cliff Pavilion', 'Wear Bay Road', 'Folkestone', 'Kent', 'CT19 6BL', '', '5', 'fhrs_5_en-gb', '2018-04-04T00:00:00', '188', 'Folkestone and Hythe', 'http://www.folkestone-hythe.gov.uk', 'foodteam@folkestone-hythe.gov.uk', {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'FHRS', {'longitude': '1.195625', 'latitude': '51.083812'}, '', 4591.765489457773, False, {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalPages': 0, 'pageSize': 0, 'pageNumber': 0}, [{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/1043695'}]])
At least one document with LocalAuthorityName other than 'Dover' exists.


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [63]:
# Change the data type from String to Decimal for longitude

query = {"geocode.longitude": {"$type": "string"}}

result = mycol.update_many(query, [{"$set": {"geocode.longitude": {"$convert": {"input": {"$toDouble": "$geocode.longitude"}, "to": "decimal"}}}}])

print(result.modified_count)

0


In [65]:
# Change the data type from String to Decimal for latitude
query = {"geocode.latitude": {"$type": "string"}}

result = mycol.update_many(query, [{"$set": {"geocode.latitude": {"$convert": {"input": {"$toDouble": "$geocode.latitude"}, "to": "decimal"}}}}])

print(result.modified_count)


38785


In [67]:
# Check that the coordinates are now numbers
query = {"geocode.longitude": {"$type": "decimal"}}
check_decimals = mycol.count_documents(query)
print('Longitude count in decimals: ',check_decimals)


query = {"geocode.latitude": {"$type": "decimal"}}
check_decimals = mycol.count_documents(query)
print('latitude count in decimals: ',check_decimals)


Longitude count in decimals:  38784
latitude count in decimals:  38785
